# Find your Next Job with TigerGraph

## Create your TigerGraph Graph

### Graph

In [ ]:
!pip install pyTigerGraph

In [ ]:
import pyTigerGraph as tg 

conn = tg.TigerGraphConnection(host="https://SUBDOMAIN.i.tgcloud.io/", password="PASSWORD")

In [ ]:
# NOTE: ONLY UNCOMMENT FOR EMERGENCIES!! IT WILL DELETE EVERYTHING FROM YOUR GRAPH!

# print(conn.gsql('''DROP ALL'''))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph LegoGraph, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 7.410 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.094 seconds
Everything is dropped.


In [ ]:
print(conn.gsql('''
CREATE VERTEX Job_Option(PRIMARY_ID job_id INT, job_link STRING, num_skills INT, num_reviews DOUBLE, num_stars DOUBLE)
CREATE VERTEX Job_Type(PRIMARY_ID job_type_id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Job_Title(PRIMARY_ID job_title_id STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Salary_Range(PRIMARY_ID salary_range STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Company(PRIMARY_ID company_name STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX Skill(PRIMARY_ID skill STRING) WITH PRIMARY_ID_AS_ATTRIBUTE="true"

CREATE UNDIRECTED EDGE JOB_OPTION_JOB_TYPE(FROM Job_Option, TO Job_Type)
CREATE UNDIRECTED EDGE JOB_OPTION_JOB_TITLE(FROM Job_Option, TO Job_Title)
CREATE UNDIRECTED EDGE JOB_TYPE_JOB_TITLE(FROM Job_Type, TO Job_Title)
CREATE UNDIRECTED EDGE JOB_OPTION_SALARY_RANGE(FROM Job_Option, To Salary_Range)
CREATE UNDIRECTED EDGE JOB_OPTION_COMPANY(FROM Job_Option, To Company)
CREATE UNDIRECTED EDGE JOB_OPTION_SKILL(FROM Job_Option, To Skill)
'''))

In [ ]:
print(conn.gsql('''
CREATE GRAPH JobGraph(
  Job_Option, Job_Type, Job_Title, Salary_Range, Company, Skill,
  JOB_OPTION_JOB_TYPE, JOB_OPTION_JOB_TITLE, JOB_TYPE_JOB_TITLE, JOB_OPTION_SALARY_RANGE, JOB_OPTION_COMPANY, JOB_OPTION_SKILL
)
'''))

The graph JobGraph is created.


In [ ]:
conn.graphname = "JobGraph"
conn.apiToken = conn.getToken(conn.createSecret())

### Grab Kaggle Data





In [ ]:
! pip install -q kaggle

Go to your Settings, click Account, scroll down to API. 

**Press "Expire" then "Create New Token". It will download a kaggle.json; upload it to the selector below.**



In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"genericp3rson","key":"17ef90b08777c6862bf7cb65191dae17"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              234KB  2021-07-24 09:20:05           9826  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6049  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2238  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2494  
fatiimaezzahra/famous-iconic-women                          Famous Iconic Wo

In [ ]:
! kaggle datasets download -d elroyggj/indeed-dataset-data-scientistanalystengineer
! unzip indeed-dataset-data-scientistanalystengineer.zip -d data
! ls data

  0% 0.00/5.05M [00:00<?, ?B/s]
100% 5.05M/5.05M [00:00<00:00, 46.5MB/s]
Archive:  indeed-dataset-data-scientistanalystengineer.zip
  inflating: train/dataset description.txt  
  inflating: train/indeed_job_dataset.csv  
'dataset description.txt'   indeed_job_dataset.csv


### Loading Data

In [ ]:
import pandas as pd 

data = pd.read_csv("data/indeed_job_dataset.csv") 

In [ ]:
data.head()

,Unnamed: 0,Job_Title,Link,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Description,Location,Company_Revenue,Company_Employees,Company_Industry,python,sql,machine learning,r,hadoop,tableau,sas,spark,java,Others,CA,NY,VA,TX,MA,IL,WA,MD,DC,NC,Other_states,Consulting and Business Services,Internet and Software,Banks and Financial Services,Health Care,Insurance,Other_industries
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,1.0,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...",MO,More than $10B (USD),"10,000+",Health Care,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,15.0,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\...",TX,NaN,NaN,NaN,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,1.0,"[<ul><li>Validate, analyze, and conduct statis...",OR,NaN,NaN,NaN,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,30.0,"[<p>Full time</p>, <p>Washington, DC metro are...",DC,NaN,NaN,Government,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,30.0,[<ul><li>Assist in consultations with business...,TX,NaN,"Less than 10,000",Banks and Financial Services,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
data["id_val"] = [val for val in data["Unnamed: 0"]]
data = data.drop("Unnamed: 0", 1)

In [ ]:
data["id"]

0          0
1          1
2          2
3          3
4          4
        ... 
5710    5710
5711    5711
5712    5712
5713    5713
5714    5714
Name: id, Length: 5715, dtype: int64

In [ ]:
data.head()

,Job_Title,Link,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Description,Location,Company_Revenue,Company_Employees,Company_Industry,python,sql,machine learning,r,hadoop,tableau,sas,spark,java,Others,CA,NY,VA,TX,MA,IL,WA,MD,DC,NC,Other_states,Consulting and Business Services,Internet and Software,Banks and Financial Services,Health Care,Insurance,Other_industries,id,id_val
0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,1.0,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...",MO,More than $10B (USD),"10,000+",Health Care,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,15.0,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\...",TX,NaN,NaN,NaN,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,1.0,"[<ul><li>Validate, analyze, and conduct statis...",OR,NaN,NaN,NaN,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,2
3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,30.0,"[<p>Full time</p>, <p>Washington, DC metro are...",DC,NaN,NaN,Government,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3,3
4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,30.0,[<ul><li>Assist in consultations with business...,TX,NaN,"Less than 10,000",Banks and Financial Services,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,4,4


In [ ]:
data["Link"] = ["No Link" if type(i) != str else i for i in data["Link"]]
data["No_of_Skills"] = [int(i) for i in data["No_of_Skills"]]
data["No_of_Reviews"] = [float(i) for i in data["No_of_Reviews"]]
data["No_of_Stars"] = [int(i) for i in data["No_of_Stars"]]

In [ ]:
conn.upsertVertexDataFrame(data, "Job_Option", "id", attributes={"job_id": "id", "job_link": "Link", "num_skills": "No_of_Skills", "num_reviews": "No_of_Reviews", "num_stars": "No_of_Stars"})

In [ ]:
data["Company"] = ["No Company Found" if type(i) != str else i for i in data["Company"]]

In [ ]:
conn.upsertVertexDataFrame(data, "Job_Type", "Job_Type", attributes={"job_type_id": "Job_Type"})
conn.upsertVertexDataFrame(data, "Job_Title", "Job_Title", attributes={"job_title_id": "Job_Title"})
conn.upsertVertexDataFrame(data, "Salary_Range", "Queried_Salary", attributes={"salary_range": "Queried_Salary"})
conn.upsertVertexDataFrame(data, "Company", "Company", attributes={"company_name": "Company"})

conn.upsertEdgeDataFrame(data, "Job_Option", "JOB_OPTION_SALARY_RANGE", "Salary_Range", "Job_Type", "Queried_Salary", attributes={})
conn.upsertEdgeDataFrame(data, "Job_Option", "JOB_OPTION_COMPANY", "Company", "Job_Type", "Job_Title", attributes={})
conn.upsertEdgeDataFrame(data, "Job_Type", "JOB_TYPE_JOB_TITLE", "Job_Title", "Job_Type", "Job_Title", attributes={})
conn.upsertEdgeDataFrame(data, "Job_Option", "JOB_OPTION_JOB_TITLE", "Job_Title", "id", "Job_Title", attributes={})
conn.upsertEdgeDataFrame(data, "Job_Option", "JOB_OPTION_JOB_TYPE", "Job_Type", "id", "Job_Type", attributes={})

5715

In [ ]:
for job in range(len(data["Skill"])):
  if type(data["Skill"][job]) == str:
      for skill in eval(data["Skill"][job]):
          conn.upsertVertex("Skill", skill, attributes={"skill": skill})
          conn.upsertEdge("Job_Option", job, "JOB_OPTION_SKILL", "Skill", skill)

## Query the Graph

In [ ]:
conn.gsql('''
USE GRAPH JobGraph

DROP QUERY topSkills

CREATE QUERY topSkills () FOR GRAPH JobGraph {
  SumAccum<INT> @connectedJobs;

  Seed = {Job_Option.*};

  Res = SELECT tgt FROM Seed:s - (JOB_OPTION_SKILL:e) - Skill:tgt
        ACCUM tgt.@connectedJobs+=1
        ORDER BY tgt.@connectedJobs DESC
        LIMIT 5;

  PRINT Res;
}

INSTALL QUERY topSkills
''')

[                                                                                     ] 0% (0/1)
[                                                                                     ] 0% (0/1)
[========================================                                             ] 47% (0/1)
[=================================================================================    ] 95% (0/1)
[=====================================================================================] 100% (1/1)



'Using graph \'JobGraph\'\nThe query topSkills is dropped.\nThe query topSkills has been added!\nStart installing queries, about 1 minute ...\ntopSkills query: curl -X GET \'https://127.0.0.1:9000/query/JobGraph/topSkills\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\n'

In [ ]:
conn.runInstalledQuery("topSkills")

In [ ]:
conn.gsql('''
USE GRAPH JobGraph

DROP QUERY topSkillsPerSalaryRange

CREATE QUERY topSkillsPerSalaryRange(Vertex<Salary_Range> salaryRange) FOR GRAPH JobGraph {
  SumAccum<INT> @connectedJobs;

  Seed = {salaryRange};
  
  Jobs = SELECT tgt FROM Seed:s - (JOB_OPTION_SALARY_RANGE:e) - Job_Option:tgt;

  Res = SELECT tgt FROM Jobs:s - (JOB_OPTION_SKILL:e) - Skill:tgt
        ACCUM tgt.@connectedJobs+=1
        ORDER BY tgt.@connectedJobs
        LIMIT 5;

  PRINT Res;
}

INSTALL QUERY topSkillsPerSalaryRange
''')

[                                                                                     ] 0% (0/1)
[                                                                                     ] 0% (0/1)
[========================================                                             ] 47% (0/1)
[=================================================================================    ] 95% (0/1)
[=====================================================================================] 100% (1/1)



'Using graph \'JobGraph\'\nQuery topSkillsPerSalaryRange could not be found.\nThe query topSkillsPerSalaryRange has been added!\nStart installing queries, about 1 minute ...\ntopSkillsPerSalaryRange query: curl -X GET \'https://127.0.0.1:9000/query/JobGraph/topSkillsPerSalaryRange?salaryRange=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\n'

In [ ]:
conn.runInstalledQuery("topSkillsPerSalaryRange", {"salaryRange": "<80000"})

[{'Res': []}]

# Congrats!

Congrats! You've now created a job graph with TigerGraph and queried it to gain some insights!

If you have any questions, feel free to ask in [the TigerGraph Discord](https://discord.gg/gRHWBZNpxW) or in [the Community Page](https://community.tigergraph.com/).